In [36]:
import json
import random
import os

In [41]:
def to_data(destPath,originFilePath,M=5,valid=1,test=2,seed=1):
    random.seed(seed)
    with open(originFilePath,'r',encoding='utf8') as f:
        origin_data = json.load(f)
    print('数据项共{}条'.format(len(origin_data)))
    entity_set = set()
    train_file = open(os.path.join(destPath,'train.char.bmes'),'w',encoding='utf8')
    valid_file = open(os.path.join(destPath,'valid.char.bmes'),'w',encoding='utf8')
    test_file = open(os.path.join(destPath,'test.char.bmes'),'w',encoding='utf8')
    train_count = 0
    test_count = 0
    valid_count = 0
    for data in origin_data:
        f = train_file
        train_count += 1
        if random.randint(0,M) == test:
            f = test_file
            train_count -= 1
            test_count += 1
        elif random.randint(0,M) == valid:
            f = valid_file
            train_count -= 1
            valid_count += 1
        entity_names = {}
        entity_starts = {}
        for entity in data['entities']:
            name,label = entity.split('-')
            entity_set.add(label)
            entity_names[name] = label
            entity_starts[name[0]] = name
        index = 0
        while index < len(data['text']):
            if data['text'][index] in entity_starts:
                start = data['text'][index]
                end = index + len(entity_starts[start])
                if entity_starts[start] == data['text'][index:end]:
                    jndex = 0
                    for ch in entity_starts[start]:
                        if jndex == 0:   
                            f.write('{} B-{}\n'.format(ch,entity_names[entity_starts[start]]))
                        elif jndex == end-index-1:
                            f.write('{} E-{}\n'.format(ch,entity_names[entity_starts[start]]))
                        else:
                            f.write('{} M-{}\n'.format(ch,entity_names[entity_starts[start]]))
                        jndex += 1
                    index = end
                    continue
            f.write('{} O\n'.format(data['text'][index]))
            index += 1
        f.write('\n')
    train_file.close()
    test_file.close()
    valid_file.close()
    print('实体类型共{}项：'.format(len(entity_set)))
    print(entity_set)
    print('训练样本共{}项，测试样本共{}项，验证样本共{}项'.format(train_count,test_count,valid_count))

In [42]:
to_data('datasets','./datasets/bmes_train.json')

数据项共4919条
实体类型共10项：
{'PER', 'RIV', 'LAK', 'RES', 'ORG', 'HYD', 'DAM', 'OTH', 'LOC', 'TER'}
训练样本共3431项，测试样本共824项，验证样本共664项
